In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression

i too df from previous eda file.

In [ ]:
X = df[['year', 'month', 'day', 'hour', '15min_per_hour', 'peek_traffic_hours', 'high_traffic_zone'	]]
y = df['demand_15min_zone']


In [ ]:
df

,cluster,year,month,day,hour,weekday,15min_per_hour,demand_15min_zone,peek_traffic_hours,high_traffic_zone
0,0,2022,1,1,13,5,0,1,0,1
1,0,2022,1,1,16,5,2,1,1,1
2,0,2022,1,1,18,5,2,1,1,1
3,0,2022,1,1,21,5,1,1,1,1
4,0,2022,1,1,22,5,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
87136,34,2023,7,31,8,0,2,2,1,1
87137,34,2023,7,31,16,0,1,1,1,1
87138,34,2023,7,31,17,0,2,1,1,1
87139,34,2023,7,31,22,0,3,1,1,1


In [ ]:

X = df[['year', 'month', 'hour', '15min_per_hour', 'peek_traffic_hours']]
Y = df['demand_15min_zone']


i dont know much about time series
so i Implemented other models

In [338]:
import time
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class ModelEvaluator:
    def __init__(self, df):
        self.df = df
        self.X = df[['year', 'month', 'day', 'hour', '15min_per_hour', 'peek_traffic_hours', 'high_traffic_zone']]
        self.y = df['demand_15min_zone']
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)
        self.scaler = StandardScaler()
        self.X_train_scaled = self.scaler.fit_transform(self.X_train)
        self.X_test_scaled = self.scaler.transform(self.X_test)
        self.models = {
            'Linear Regression': LinearRegression(),
            'XGBoost': XGBRegressor(),
            'Random Forest': RandomForestRegressor(),
            'LightGBM': lgb.LGBMRegressor(),
            'DNN': self.build_dnn_model()
        }

    def build_dnn_model(self):
        model = keras.Sequential([
            layers.Dense(64, activation='relu', input_shape=(self.X_train_scaled.shape[1],)),
            layers.Dense(32, activation='relu'),
            layers.Dense(1)
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        return model

    def train_models(self):
        training_times = {}
        for name, model in self.models.items():
            start_time = time.time()
            if name == 'DNN':
                model.fit(self.X_train_scaled, self.y_train, epochs=20, batch_size=32, validation_data=(self.X_test_scaled, self.y_test), verbose=0)
            else:
                model.fit(self.X_train_scaled, self.y_train)
            end_time = time.time()
            training_times[name] = {'time_taken': end_time - start_time, 'params': self.get_model_params(model)}

        return training_times

    def get_model_params(self, model):
        if 'DNN' in str(type(model)):
            return model.count_params()
        # Provide information about parameters for non-neural network models
        elif 'RandomForestRegressor' in str(type(model)):
            return {'n_estimators': model.n_estimators, 'max_depth': model.max_depth}
        elif 'XGBRegressor' in str(type(model)):
            return {'n_estimators': model.n_estimators, 'max_depth': model.max_depth}
        elif 'LGBMRegressor' in str(type(model)):
            return {'n_estimators': model.n_estimators, 'max_depth': model.max_depth}
        elif 'LinearRegression' in str(type(model)):
            return {}

    def evaluate_models(self):
        metrics = {}
        for name, model in self.models.items():
            if name == 'DNN':
                predictions = model.predict(self.X_test_scaled).flatten()
            else:
                predictions = model.predict(self.X_test_scaled)

            rmse = np.sqrt(mean_squared_error(self.y_test, predictions))
            mae = mean_absolute_error(self.y_test, predictions)
            metrics[name] = {'RMSE': rmse, 'MAE': mae}
            print(f'{name} - RMSE: {rmse:.4f}, MAE: {mae:.4f}')

        return metrics


# Example usage
df = df= demand_by_time_zone
model_evaluator = ModelEvaluator(df)
training_info = model_evaluator.train_models()
metrics = model_evaluator.evaluate_models()

# Display training times and parameters
for name, info in training_info.items():
    print(f'{name} - Time taken: {info["time_taken"]:.2f} seconds')
    print(f'{name} - Model Parameters: {info["params"]}')

# Note: For non-DNN models, the 'params' key will contain information about the model parameters.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003863 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 84
[LightGBM] [Info] Number of data points in the train set: 69712, number of used features: 7
[LightGBM] [Info] Start training from score 1.136705
Linear Regression - RMSE: 0.3894, MAE: 0.2305
XGBoost - RMSE: 0.3698, MAE: 0.1997
Random Forest - RMSE: 0.3947, MAE: 0.2016
LightGBM - RMSE: 0.3678, MAE: 0.1974
545/545 [==============================] - 1s 1ms/step
DNN - RMSE: 0.3688, MAE: 0.2164
Linear Regression - Time taken: 0.03 seconds
Linear Regression - Model Parameters: {}
XGBoost - Time taken: 0.52 seconds
XGBoost - Model Parameters: {'n_estimators': None, 'max_depth': None}
Random Forest - Time taken: 14.97 seconds
Random Forest - Model Parameters: {'n_estimators': 100, 'max_depth': None}
LightGBM - Time taken: 0.43 seconds
Light

In [3]:
import pandas as pd
from IPython.display import display, HTML

# Results from your model evaluation
results = {
    'Linear Regression': {'RMSE': 0.3894, 'MAE': 0.2305, 'Time taken': 0.03, 'Model Parameters': {}},
    'XGBoost': {'RMSE': 0.3698, 'MAE': 0.1997, 'Time taken': 0.52, 'Model Parameters': {'n_estimators': None, 'max_depth': None}},
    'Random Forest': {'RMSE': 0.3947, 'MAE': 0.2016, 'Time taken': 14.97, 'Model Parameters': {'n_estimators': 100, 'max_depth': None}},
    'LightGBM': {'RMSE': 0.3678, 'MAE': 0.1974, 'Time taken': 0.43, 'Model Parameters': {'n_estimators': 100, 'max_depth': -1}},
    'DNN': {'RMSE': 0.3688, 'MAE': 0.2164, 'Time taken': 142.94, 'Model Parameters': None}
}

# Convert the results dictionary to a DataFrame
df_results = pd.DataFrame(results).T

# Display the HTML table in the Jupyter Notebook
display(HTML(df_results.to_html()))


,RMSE,MAE,Time taken,Model Parameters
Linear Regression,0.3894,0.2305,0.03,{}
XGBoost,0.3698,0.1997,0.52,"{'n_estimators': None, 'max_depth': None}"
Random Forest,0.3947,0.2016,14.97,"{'n_estimators': 100, 'max_depth': None}"
LightGBM,0.3678,0.1974,0.43,"{'n_estimators': 100, 'max_depth': -1}"
DNN,0.3688,0.2164,142.94,NaN
